In [1]:
import os

In [2]:
%pwd

'd:\\ML_Projects\\Resume Project\\Project_6_insurance_claim_prediction\\reasearch'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\ML_Projects\\Resume Project\\Project_6_insurance_claim_prediction'

In [5]:
# Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float # params.yaml  
    l1_ratio: float # params.yaml
    target_column: str # Schema.yaml




In [6]:
# prepare confuguration manager class
from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml,create_directories

In [11]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainingConfig:
        config =  self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainingConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path =  config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha, # params.yaml  
            l1_ratio = params.l1_ratio, # params.yaml
            target_column = schema.name # Schema.yaml
        )

        return model_trainer_config

In [12]:
import os
import pandas as pd
from src.mlproject.logging import logger
from sklearn.linear_model import ElasticNet
import joblib

In [17]:
class ModelTrainer:
    def __init__(self,config: ModelTrainingConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        
        train_X = train_data.drop([self.config.target_column],axis=1)
        test_X = test_data.drop([self.config.target_column],axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]
        print(train_X)


        lr = ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio,random_state=42)
        lr.fit(train_X,train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir,self.config.model_name))

In [18]:
# Pipeline
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-04-10 01:30:30,048: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-10 01:30:30,053: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-10 01:30:30,053: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-10 01:30:30,070: INFO: common: Created directory at: artifacts]


[2025-04-10 01:30:30,107: INFO: common: Created directory at: artifacts/model_trainer]
      num__age  num__bmi  num__children  cat__sex_female  cat__sex_male  \
0         34.0    30.210            1.0              1.0            0.0   
1         54.0    31.600            0.0              0.0            1.0   
2         26.0    31.065            0.0              0.0            1.0   
3         39.0    24.510            2.0              0.0            1.0   
4         24.0    23.210            0.0              1.0            0.0   
...        ...       ...            ...              ...            ...   
2074      59.0    27.500            1.0              0.0            1.0   
2075      53.0    31.160            1.0              0.0            1.0   
2076      38.0    30.690            1.0              1.0            0.0   
2077      18.0    31.350            0.0              1.0            0.0   
2078      23.0    28.310            0.0              1.0            0.0   

      cat__s